In [ ]:
"""
automating a one minute trade for the user 
i.e. after 1-minute, 
if candlestick is green(ltp: open<close) then 'Buy' 
or if candlestick is red(ltp: open>close) then 'sell'
"""

In [ ]:
#automating login and access token generation using selenium

import requests
from requests_oauthlib import OAuth2Session
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException

options = webdriver.ChromeOptions()
# options.add_argument('headless')
driver = webdriver.Chrome(options=options)

web_url = 'https://ant.aliceblueonline.com'
client_id = 'HO***' #enter alice client id
client_secret = 'Avrt7mMhEjs8hZFb3IKZybuDhsxwnN************************' #your generated client secret
redirect_uri = 'http://127.0.0.1/'
authorization_base_url = f'{web_url}/oauth2/auth'
token_url = f'{web_url}/oauth2/token'
scope=['orders']
oauth = OAuth2Session(client_id, redirect_uri=redirect_uri, scope=scope)
authorization_url, _state = oauth.authorization_url(authorization_base_url, access_type="authorization_code")

driver.get(authorization_url)

waitingEle = WebDriverWait(driver, 100).until(EC.presence_of_element_located((By.NAME, 'client_id')))
clientId = driver.find_element_by_name('client_id')
clientId.send_keys('$$$$$') #enter your client_id
password = driver.find_element_by_name('password')
password.send_keys('#####') #enter your password
button = driver.find_element_by_class_name('container').find_element_by_xpath(".//div[1]/button")
button.click()

waitingEle = WebDriverWait(driver, 100).until(EC.presence_of_element_located((By.NAME, 'answer1')))
a1 = driver.find_element_by_name('answer1')
a1.send_keys('$$$$$') #answer 1st security ques
a2 = driver.find_element_by_name('answer2')
a2.send_keys('$$$$$') #answer 2nd security ques
button = driver.find_element_by_class_name('container').find_element_by_xpath(".//div[1]/button")
button.click()
wait = WebDriverWait(driver, 100)
wait.until(lambda driver: driver.current_url.startswith != "http://127.0.0.1")

newUrl = driver.current_url
# print(newUrl)
driver.close()
authUrl = 'https' + newUrl[4:]

# print(clientId.get_attribute("style"))

oauth = OAuth2Session(client_id, redirect_uri=redirect_uri, scope=scope)
token = oauth.fetch_token(token_url, authorization_response=authUrl, client_secret=client_secret)
access_token = token['access_token']
print(access_token)


In [ ]:
'''
1) creating 'alice blue' object for using aliceblue api 
2) retrieving master contracts
'''
import os
from requests_oauthlib import OAuth2Session
from flask import Flask, request, redirect
import json
from alice_blue import *
import datetime 
import time

os.environ['OAUTHLIB_INSECURE_TRANSPORT'] = '1'
username = '$$$$$$$' #alice username
password = '*********' #password
alice = AliceBlue(username=username, password=password, access_token=access_token)
alice = AliceBlue(username=username, password=password, access_token=access_token, master_contracts_to_download=['NSE','NFO','MCX'])
print('done')

In [ ]:
'''Global variables'''
global today
global startTime
global endTime
global marketClose
global values

In [ ]:
''' Initializing global variable according to user specified 1-minute '''
today = datetime.datetime.now()
startTime = datetime.datetime(today.year,today.month,today.day,hour = 9, minute = 15, second = 0)
endTime = datetime.datetime(today.year,today.month,today.day,hour = 9, minute = 16, second = 0)
marketClose = datetime.datetime(today.year,today.month,today.day,hour = 15, minute = 15, second = 0)
symbols = open("input.txt",'r').read().split('\n')
symbolData = {name:{'ltp':[],'o':0,'h':0,'l':0,'c':0} for name in symbols}
symbolData

In [ ]:
'''
Using the LTP retrievation api event handler with checking the condition of the candlestick,
and performing trade according to that
'''
socket_opened = False
def event_handler_quote_update(message):
    global symbolData
    if datetime.datetime.now().time()<endTime:
        symbolData[message['instrument'].symbol]['ltp'].append(message['ltp'])
        print(f"quote update {message}")
        print(symbolData)
    else:
        opening = symbolData[message['instrument'].symbol]['ltp'][0]
        closing = symbolData[message['instrument'].symbol]['ltp'][-1]
        high = max(symbolData[message['instrument'].symbol]['ltp'])
        low = min(symbolData[message['instrument'].symbol]['ltp'])
        symbolData[message['instrument'].symbol]['o'] = opening
        symbolData[message['instrument'].symbol]['h'] = high
        symbolData[message['instrument'].symbol]['l'] = low
        symbolData[message['instrument'].symbol]['c'] = closing
        #return
        print(symbolData)
        if opening < closing:
            print("Candlestick is Green...Buy")
            # TransactionType.Buy, OrderType.Market, ProductType.Delivery
            print ("%%%%%%%%%%%%%%%%%%%%%%%%%%%%1%%%%%%%%%%%%%%%%%%%%%%%%%%%%%")
            print(
                   alice.place_order(transaction_type = TransactionType.Buy,
                                     instrument = alice.get_instrument_by_symbol('NSE', message['instrument'].symbol),
                                     quantity = 1,
                                     order_type = OrderType.Market,
                                     product_type = ProductType.Delivery,
                                     price = 0.0,
                                     trigger_price = None,
                                     stop_loss = low,
                                     square_off = None,
                                     trailing_sl = None,
                                     is_amo = False)
            )
        elif opening > closing:
            print("Candlestick is Red...Sell")
            # TransactionType.Sell, OrderType.Market, ProductType.Delivery
            print ("%%%%%%%%%%%%%%%%%%%%%%%%%%%%1%%%%%%%%%%%%%%%%%%%%%%%%%%%%%")
            print(
                   alice.place_order(transaction_type = TransactionType.Sell,
                                     instrument = alice.get_instrument_by_symbol('NSE', message['instrument'].symbol),
                                     quantity = 1,
                                     order_type = OrderType.Market,
                                     product_type = ProductType.Delivery,
                                     price = 0.0,
                                     trigger_price = None,
                                     stop_loss = high,
                                     square_off = None,
                                     trailing_sl = None,
                                     is_amo = False)
            )
    
    
def open_callback():
    global socket_opened
    socket_opened = True

alice.start_websocket(subscribe_callback=event_handler_quote_update,
                      socket_open_callback=open_callback,
                      run_in_background=True)

alice.subscribe([alice.get_instrument_by_symbol('NSE', name) for name in symbols],LiveFeedType.MARKET_DATA)

In [ ]:
#once trade time is over, unsubscribe from the market updates 
alice.unsubscribe([alice.get_instrument_by_symbol('NSE', name) for name in symbols, LiveFeedType.MARKET_DATA)